## Example: Twitter API in Python, with OAuth1 authentication

<img src="images/Python_Twitter.jpg" width="300px">

The guide below shows the steps of registering an application that you can integrate with **Twitter**. Registering the application is the first step in developing an application that is integrated into its users' social graph.

## Register a new application

All **Twitter** users are potentially **Twitter** application developers. Simply visit https://apps.twitter.com/ and sign in with your **Twitter** credentials.

Click the "Create a new application" button near the top. A new page with the **_Create an application_** form requires basic information about your application.

<img src="images/create_an_application_1.jpg">

- In the **_Name_** field name your application in 32 characters or fewer. 
- In the **_Description_** field describe your application in 10 to 200 characters.
- In the **_Website_** field, you can give a pointer to your website. (It does not really matter, as we will not be using that functionality yet).
- The **_Callback URL_** leave it empty for now.

The **Developer Agreement** section outlines rules you must agree to follow if you build an application that uses **Twitter’s** API. If you agree to the rules, check "Yes, I agree".

<img src="images/create_an_application_2.jpg">

After Captcha challenge click "Create your **Twitter** application" to complete the form and go to the application settings page.

In the opened window you’ll then be presented with lots of information. The main fields to note are **_Consumer key_** and **_Consumer secret_**. These values are your application’s credentials for **Twitter**. You need them to do almost anything with **Twitter**, including going through the OAuth authorization flow and working with **Twitter’s** REST API. But we’re not quite done yet. We now need to authorise the **Twitter** app for your **Twitter** account. To do this you should select "Key and Access Tokens" menu button and create your access token. This **access token** will allow your **Twitter** application to read **Twitter** information. You’ll be able to get data of your tweets, mentions, lists and more. 

<img src="images/create_an_application_3.jpg">

As a result you will see the window with consumer and token keys for your application 

<img src="images/create_an_application_4.jpg">

Above the four fields are highlighted. You will need these long horrible strings of characters for your **Twitter** app. It goes without saying that you should keep these secret. If anyone was to get these keys, they could effectively access your **Twitter** account.

However, if you want to do more advanced stuff like sending tweets or deleting, you’ll need to change your access type in the "Permission" menu window. Change the access type to "Read and Write" to be able to read **Twitter** data and send tweets and select "Read, Write and access direct messages" if you want to also have access to your direct messages.

---
## Getting access to Twitter APIs using main Python tools
---

Many web services including **Twitter** require authentication, and there are many different types. 

Python library `requests` provides a pythonic way to make complex HTTP requests, and handles difficult tasks like authentication.

A common form of authentication for several web APIs is OAuth. The `requests-oauthlib` library allows `requests`'s users to easily make OAuth authenticated requests.

In [4]:
import requests
from requests_oauthlib import OAuth1

import json

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
consumer_key = 'spQSlEddMBQPkbBg9ncduA3Nx'
consumer_secret = 'tIunlVXi4eD69EfgJDnqSX6s8EaYobobtFi1HpGnWvugKonzQZ'

access_token = '16976496-hhbdqyfoVqgYbFmVz076IwWxa4vM58my0v1nJNlW0'
access_secret = '3aTWUUdNMTVWDBjxCOApzRdSiiySFM4yaZcGJ1YYaub75'

# You will authenticate yourself using OAuth1 object
auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
print auth

In [6]:
# If the authentication was successful, you should see the name of the account print out
url_0 = 'https://api.twitter.com/1.1/account/verify_credentials.json'
res = requests.get(url_0, auth=auth)

print "My name is", res.json()["name"]

My name is Panos Ipeirotis


### Example 1: Read Tweets from Your Homepage

The list of all available **Twitter** API you can find here https://dev.twitter.com/rest/public. Let's use `home_timeline` that returns a collection of the most recent tweets and retweets posted by the authenticating user and the users they follow.

In [9]:
url_1 = 'https://api.twitter.com/1.1/statuses/home_timeline.json'
res = requests.get(url_1, auth=auth, params={"count": 20})

# The res object encapsulates the "response" of the server. Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 403 means that the server understood the request, but is refusing to fulfill it
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong

print res, "Status code:", res.status_code
# Let's see how looks the url
print res.url
print "Content type:", res.headers['content-type'], '\n'

tweets = res.json()
if res.status_code == 200:
    for tweet in tweets:
        print tweet['text']
else:   # You have no tweets
    print tweets

<Response [200]> Status code: 200
https://api.twitter.com/1.1/statuses/home_timeline.json?count=20
Content type: application/json;charset=utf-8 

That simple smile when you know the right answer, but saying it is not the right thing to do.
Worth reading: I Was One of the Most Ardent Hillary Haters on the Planet--Until I Read Her Emails https://t.co/JYAcofZ9bx
@adityagp @marcua since you'll have insights that other people won't, because you felt these experiences firsthand
@adityagp @marcua this experience will undoubtedly make you more creative &amp; innovative in your future research projects
.@marcua @pgbovine thanks -- it's exhausting (a labor of love) but well worth the effort!
RT @ana_gram_: Come see moral pop-out for yourself! You can borrow my stereoscope - poster behind room 3 #spsp2016 https://t.co/Yr5YFE8wH1
RT @daveambrose: .@joshk on “investing in chefs" and “not raw ingredients" https://t.co/ypnzdWyosw https://t.co/2P8rnOBdlV
Just watçhed the @VICE special report on prison

### Example 2: Search tweets by key words

**Twitter** employs a special query language.  For example, the query _"vacation?"_ will return tweets that contain the word "vacation" and are phrased as a question OR the query _"summer"_ will return the word "summer" with a positive attitude. [Check out more examples here](https://dev.twitter.com/rest/public/search).

We can mine tweets using either **_search_** or **_stream_**.

>The key difference between **_stream_** and **_search_** is that **_stream_** provides new data as it comes in, while **_search_** can be used to query old data. The **_search_** API is more powerful for queries, and provides faster access to a wide-range of data.

Let's search for a single tweet about "vacation", phrased as a question. Note if request phrase contains URI characters you should [encode](https://en.wikipedia.org/wiki/Percent-encoding) these queries before making the request, because these characters can play role of keys in an URL. Particularly, the question mark "?" is encoded as "%3F".

In [11]:
url_2 = 'https://api.twitter.com/1.1/search/tweets.json?q=stern,nyu'
res = requests.get(url_2, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

tweets = res.json()
tweets

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=stern,nyu


{u'search_metadata': {u'completed_in': 0.115,
  u'count': 15,
  u'max_id': 692904504090595328,
  u'max_id_str': u'692904504090595328',
  u'next_results': u'?max_id=692780365430284288&q=stern%2Cnyu&include_entities=1',
  u'query': u'stern%2Cnyu',
  u'refresh_url': u'?since_id=692904504090595328&q=stern%2Cnyu&include_entities=1',
  u'since_id': 0,
  u'since_id_str': u'0'},
 u'statuses': [{u'contributors': None,
   u'coordinates': None,
   u'created_at': u'Fri Jan 29 02:58:01 +0000 2016',
   u'entities': {u'hashtags': [{u'indices': [78, 93],
      u'text': u'bizhumanrights'}],
    u'symbols': [],
    u'urls': [{u'display_url': u'ow.ly/XCULf',
      u'expanded_url': u'http://ow.ly/XCULf',
      u'indices': [139, 140],
      u'url': u'https://t.co/XfRcutgNvk'}],
    u'user_mentions': [{u'id': 920875687,
      u'id_str': u'920875687',
      u'indices': [3, 17],
      u'name': u'CORE Coalition',
      u'screen_name': u'corecoalition'},
     {u'id': 2205282823,
      u'id_str': u'2205282823',


Please pay attention on how much information each tweet contains.

In [13]:
# And let's see tweets text directly
for num, tweet in enumerate(tweets['statuses']):
    print 'Tweet #{0}\t{1}\t{2}\n{3}\n'.format(num+1, tweet['created_at'], tweet['retweet_count'], tweet['text'].encode("utf-8"))

Tweet #1	Fri Jan 29 02:58:01 +0000 2016	3
RT @corecoalition: Interesting from @NYUSternBHR on barriers to incorporating #bizhumanrights learning in university business courses https…

Tweet #2	Fri Jan 29 01:56:04 +0000 2016	0
Asy. Power MEM: ELECTRICITE ET EAUX DE MADAGASCAR ASY. POWER MEM VOLATILITY GRAPH. Volatility Prediction for F... https://t.co/cakH3yEoIf

Tweet #3	Fri Jan 29 01:56:03 +0000 2016	0
garch: ELECTRICITE ET EAUX DE MADAGASCAR GARCH VOLATILITY GRAPH. Volatility Prediction for Friday, January 29,... https://t.co/LqOiOErA13

Tweet #4	Fri Jan 29 01:17:17 +0000 2016	1
RT @nyupoly: Download speeds 1000x faster than today’s current mobile phones? Yes, please. https://t.co/jz0Y99hjDX

Tweet #5	Fri Jan 29 00:40:44 +0000 2016	1
RT @nikimari: @aghose on imperative that SEC considers #crowdfunding issues &amp; provides guidance to investors and entrepreneurs. https://t.c…

Tweet #6	Thu Jan 28 23:51:22 +0000 2016	1
Download speeds 1000x faster than today’s current mobile phones? Y

In [18]:
import pandas as pd
df = pd.DataFrame(tweets['statuses'])
df

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Fri Jan 29 02:58:01 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 92...",0,False,None,692904504090595328,692904504090595328,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,3,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @corecoalition: Interesting from @NYUSternB...,False,"{u'follow_request_sent': False, u'has_extended..."
1,None,None,Fri Jan 29 01:56:04 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,692888910263758850,692888910263758850,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",Asy. Power MEM: ELECTRICITE ET EAUX DE MADAGAS...,False,"{u'follow_request_sent': False, u'has_extended..."
2,None,None,Fri Jan 29 01:56:03 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,692888909412302848,692888909412302848,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",garch: ELECTRICITE ET EAUX DE MADAGASCAR GARCH...,False,"{u'follow_request_sent': False, u'has_extended..."
3,None,None,Fri Jan 29 01:17:17 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 18...",0,False,None,692879151443763200,692879151443763200,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @nyupoly: Download speeds 1000x faster than...,False,"{u'follow_request_sent': False, u'has_extended..."
4,None,None,Fri Jan 29 00:40:44 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 10...",0,False,None,692869955264520192,692869955264520192,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @nikimari: @aghose on imperative that SEC c...,False,"{u'follow_request_sent': False, u'has_extended..."
5,None,None,Thu Jan 28 23:51:22 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,692857532000243712,692857532000243712,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,NaN,"<a href=""http://bufferapp.com"" rel=""nofollow"">...",Download speeds 1000x faster than today’s curr...,False,"{u'follow_request_sent': False, u'has_extended..."
6,None,None,Thu Jan 28 22:28:30 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 16...",0,False,None,692836674267586560,692836674267586560,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,2,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @teppofelin: nice work by Pierpaolo Andrian...,False,"{u'follow_request_sent': False, u'has_extended..."
7,None,None,Thu Jan 28 20:27:55 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 16...",0,False,None,692806328540999681,692806328540999681,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,2,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @teppofelin: nice work by Pierpaolo Andrian...,False,"{u'follow_request_sent': False, u'has_extended..."
8,None,None,Thu Jan 28 19:09:34 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",1,False,None,692786614255357952,692786614255357952,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,2,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",nice work by Pierpaolo Andriani (Kedge) and Gi...,False,"{u'follow_request_sent': False, u'has_extended..."
9,None,No

In [19]:
# If we want to flatten the nested JSON entries
# we can use the json_normalize function
# http://pandas-docs.github.io/pandas-docs-travis/io.html#normalization
import pandas as pd
from pandas.io.json import json_normalize
data = json_normalize(tweets['statuses'])
df_flat = pd.DataFrame(data)
df

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Fri Jan 29 02:58:01 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 92...",0,False,None,692904504090595328,692904504090595328,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,3,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @corecoalition: Interesting from @NYUSternB...,False,"{u'follow_request_sent': False, u'has_extended..."
1,None,None,Fri Jan 29 01:56:04 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,692888910263758850,692888910263758850,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",Asy. Power MEM: ELECTRICITE ET EAUX DE MADAGAS...,False,"{u'follow_request_sent': False, u'has_extended..."
2,None,None,Fri Jan 29 01:56:03 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,692888909412302848,692888909412302848,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",garch: ELECTRICITE ET EAUX DE MADAGASCAR GARCH...,False,"{u'follow_request_sent': False, u'has_extended..."
3,None,None,Fri Jan 29 01:17:17 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 18...",0,False,None,692879151443763200,692879151443763200,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @nyupoly: Download speeds 1000x faster than...,False,"{u'follow_request_sent': False, u'has_extended..."
4,None,None,Fri Jan 29 00:40:44 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 10...",0,False,None,692869955264520192,692869955264520192,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @nikimari: @aghose on imperative that SEC c...,False,"{u'follow_request_sent': False, u'has_extended..."
5,None,None,Thu Jan 28 23:51:22 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,692857532000243712,692857532000243712,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,NaN,"<a href=""http://bufferapp.com"" rel=""nofollow"">...",Download speeds 1000x faster than today’s curr...,False,"{u'follow_request_sent': False, u'has_extended..."
6,None,None,Thu Jan 28 22:28:30 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 16...",0,False,None,692836674267586560,692836674267586560,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,2,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @teppofelin: nice work by Pierpaolo Andrian...,False,"{u'follow_request_sent': False, u'has_extended..."
7,None,None,Thu Jan 28 20:27:55 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 16...",0,False,None,692806328540999681,692806328540999681,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,2,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @teppofelin: nice work by Pierpaolo Andrian...,False,"{u'follow_request_sent': False, u'has_extended..."
8,None,None,Thu Jan 28 19:09:34 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",1,False,None,692786614255357952,692786614255357952,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,2,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",nice work by Pierpaolo Andriani (Kedge) and Gi...,False,"{u'follow_request_sent': False, u'has_extended..."
9,None,No

You may create very specialized and more concrete queries. 

Let's find 5 tweets that contains the word "python" or "IPython" near New York. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language). Additional parameters allows build very complicated search requests.

In [20]:
url_2_2 = 'https://api.twitter.com/1.1/search/tweets.json?q=python OR ipython since%3A2010-12-01 until%3A2015-12-31'

params = {
    "count": 5, 
    "show_user": False,
    "geocode": '40.7127, -74.0059, 15mi',
    "lang": 'en'
}

res = requests.get(url_2_2, auth=auth, params=params)

print res, res.status_code, res.headers['content-type']
print res.url

tweets = res.json()
for num, tweet in enumerate(tweets['statuses']):
    print 'Tweet #{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8"))

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=python%20OR%20ipython%20since%3A2010-12-01%20until%3A2015-12-31&count=5&lang=en&geocode=40.7127%2C+-74.0059%2C+15mi&show_user=False


### Example 3: Get a list of all your followers

**Twitter** GET request `followers/list` returns a cursored collection of user objects for users following the specified user. At this time, results are ordered with the most recent following first.

In [25]:
url_3 = 'https://api.twitter.com/1.1/followers/list.json'
res = requests.get(url_3, auth=auth, params=params)

print res, res.status_code, res.headers['content-type']
print res.url

followers = res.json()
followers

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/followers/list.json?count=5000


{u'next_cursor': 1523290574219221031,
 u'next_cursor_str': u'1523290574219221031',
 u'previous_cursor': 0,
 u'previous_cursor_str': u'0',
 u'users': [{u'blocked_by': False,
   u'blocking': False,
   u'contributors_enabled': False,
   u'created_at': u'Fri Feb 27 16:38:39 +0000 2009',
   u'default_profile': False,
   u'default_profile_image': False,
   u'description': u'UX Designer @detectica / George Mason Alum / LinkedIn https://t.co/lOn5P5l569 / Medium https://t.co/xBrcYTSDBP / @MattyMusicPicks / @wooliffotog',
   u'entities': {u'description': {u'urls': [{u'display_url': u'bit.ly/1PLHUd2',
       u'expanded_url': u'http://bit.ly/1PLHUd2',
       u'indices': [54, 77],
       u'url': u'https://t.co/lOn5P5l569'},
      {u'display_url': u'bit.ly/1S9gcsG',
       u'expanded_url': u'http://bit.ly/1S9gcsG',
       u'indices': [87, 110],
       u'url': u'https://t.co/xBrcYTSDBP'}]},
    u'url': {u'urls': [{u'display_url': u'matthewvoshell.com',
       u'expanded_url': u'http://www.matthewvosh

In [24]:
# And let's see only followers' name
for num, follower in enumerate(followers['users']):
    print 'My follower #{0}\t{1}'.format(num+1, follower['name'].encode("utf-8"))

My follower #1	Matthew Voshell
My follower #2	Shengli Hu
My follower #3	Yuanyang Liu
My follower #4	luiggino
My follower #5	Paris Lambert
My follower #6	Michael Feldman
My follower #7	TheVideoCorporation
My follower #8	Brennan
My follower #9	Kevin Boudreau
My follower #10	ZarMoney App
My follower #11	Andrey Seleznov
My follower #12	Nicole Mardis
My follower #13	David A Isaacs
My follower #14	MTurk Crowd
My follower #15	Business Unbound
My follower #16	Ian Dennis Miller
My follower #17	Kate Dericott
My follower #18	srinivasu
My follower #19	Daemo
My follower #20	fehmi tuncer


### Example 4:  Finding what is trending

According to the [tweepy API](http://tweepy.readthedocs.org/en/v3.5.0/api.html), we can return the top 50 trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, New York has a `WOEID` of 2459115. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in New York

In [28]:
url_4 = 'https://api.twitter.com/1.1/trends/place.json?id=2459115'
res = requests.get(url_4, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

top50_trends = res.json()
top50_trends

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/trends/place.json?id=2459115


[{u'as_of': u'2016-01-29T05:22:13Z',
  u'created_at': u'2016-01-29T05:18:18Z',
  u'locations': [{u'name': u'New York', u'woeid': 2459115}],
  u'trends': [{u'name': u'#GOPDebate',
    u'promoted_content': None,
    u'query': u'%23GOPDebate',
    u'tweet_volume': 745257,
    u'url': u'http://twitter.com/search?q=%23GOPDebate'},
   {u'name': u'#PillowTalk',
    u'promoted_content': None,
    u'query': u'%23PillowTalk',
    u'tweet_volume': 366717,
    u'url': u'http://twitter.com/search?q=%23PillowTalk'},
   {u'name': u'#CatSongs',
    u'promoted_content': None,
    u'query': u'%23CatSongs',
    u'tweet_volume': 11739,
    u'url': u'http://twitter.com/search?q=%23CatSongs'},
   {u'name': u'#ShadesOfBlue',
    u'promoted_content': None,
    u'query': u'%23ShadesOfBlue',
    u'tweet_volume': 20068,
    u'url': u'http://twitter.com/search?q=%23ShadesOfBlue'},
   {u'name': u'Clarke',
    u'promoted_content': None,
    u'query': u'Clarke',
    u'tweet_volume': 66751,
    u'url': u'http://twitt

In [27]:
# And let's see only the first 10 trends names and the respective URL
for i, trend in enumerate(top50_trends[0]['trends'][:10]):
    print '{0} - {1} - URL: {2}'.format(i+1, trend['name'], trend['url'])

1 - #GOPDebate - URL: http://twitter.com/search?q=%23GOPDebate
2 - #PillowTalk - URL: http://twitter.com/search?q=%23PillowTalk
3 - #CatSongs - URL: http://twitter.com/search?q=%23CatSongs
4 - #ShadesOfBlue - URL: http://twitter.com/search?q=%23ShadesOfBlue
5 - Clarke - URL: http://twitter.com/search?q=Clarke
6 - #SmackDown - URL: http://twitter.com/search?q=%23SmackDown
7 - Frank Luntz - URL: http://twitter.com/search?q=%22Frank+Luntz%22
8 - Paul Kantner - URL: http://twitter.com/search?q=%22Paul+Kantner%22
9 - Michael Phelps - URL: http://twitter.com/search?q=%22Michael+Phelps%22
10 - Bryce Alford - URL: http://twitter.com/search?q=%22Bryce+Alford%22


### Example 5: Streaming

**Twitter** offers a [Streaming API](https://dev.twitter.com/streaming/overview) to make it easier to query streams of tweets.  The Stream API encapsulates some pain points of REST access to ensure that Stream calls don't exceed the rate limit. These tool allows to get tweet data when it appears in real time. There are three stream types:

- `Public Streams:` Streams of public data flowthing through **Twitter**. Suitable for followign specific users, topics or for data mining.
- `User Streams:` Single-user streams. Containing roughly all of the data corresponding with a single user's view of **Twitter**. 
- `Site Streams:` The multi-user version of user streams.

>Connecting to the streaming API requires keeping a persistent HTTP connection open. In many cases this involves thinking about your application differently than if you were interacting with the REST API. An app which connects to the Streaming APIs will not be able to establish a connection in response to a user request. Instead, the code for maintaining the Streaming connection is typically run in a process separate from the process which handles HTTP requests. The streaming process gets the input tweets and performs any parsing, filtering, and/or aggregation needed before storing the result to a data store. The HTTP handling process queries the data store for results in response to user requests. While this model is more complex than the first example, the benefits from having a realtime stream of tweet data make the integration worthwhile for many types of apps.

With `requests.Response.iter_lines()` you can easily iterate over streaming APIs including the **Twitter Streaming API**. Simply set `stream` parameter to `True` and iterate over the response with `iter_lines()`. 

Depending on the search term/terms, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#iphone_ hashtag:

In [32]:
import time

url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=trump'
res = requests.get(url_5, auth=auth, stream=True)

print res, res.status_code, res.headers['content-type']
print res.url, "\n"

# Let's measure the time elapsed after the start of streaming 
start = time.time()
stop_after = 100
tweets_printed = 0
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line)
        print "Elapsed:", time.time()-start, "sec", parsed_line["created_at"], "\n", parsed_line['text'].encode("utf-8"), "\n"
        tweets_printed = tweets_printed + 1
        if tweets_printed>stop_after:
            break

 <Response [200]> 200 application/json
https://stream.twitter.com/1.1/statuses/filter.json?track=trump 

Elapsed: 0.000615119934082 sec Fri Jan 29 05:26:21 +0000 2016 
The Republican Party Can't Rid Itself Of Trump, Even When He's Not There https://t.co/aDp889EBMJ #money #dogecoin #bitcoin #news, #love, #… 

Elapsed: 0.0146601200104 sec Fri Jan 29 05:26:21 +0000 2016 
@JuiceBrennar up with the bs some foreign leaders pull, Obama is very relaxed in how he handles these issues while Trump comes with the 

Elapsed: 0.0474531650543 sec Fri Jan 29 05:26:21 +0000 2016 
RT @AnnCoulter: This is so great!  Trump is basically introducing his cabinet. 

Elapsed: 0.0599739551544 sec Fri Jan 29 05:26:21 +0000 2016 
RT @AndreaTantaros: You really do become the person you disdain. Many pundits have become far ruder &amp; nastier toward Trump than they ever a… 

Elapsed: 0.116148948669 sec Fri Jan 29 05:26:21 +0000 2016 
Our story tonight from Des Moines: Without Trump on the debate stage, @tedcruz st

---
## Exercises
---

## Exercise #1: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display, the list of all your friends and sort them by its followers amounts in descending order. 

#### HINT: `GET friends/list`  will be helpfull for you here.

## Exercise #2: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library, display the list of 5 (or less if there is not so much) most recent tweets liked by you, which were also retweeted (`retweet_count` is not equal to zero). You shoud show the date of creation, author of the tweet and its text.

#### HINT: `GET favorites/list` will be helpfull for you here

## Exercise #3: 

> Using [Twitter Streaming API](https://dev.twitter.com/streaming/overview) and `requests` Python library count how many tweets with hashtags _#twitter_, _#tweet_ and _#world_ are appearing each minute over 5 minutes and display these 5 numbers. Please display also the shortest and the longest time period between to closest tweets for all 5 measures.

#### HINT: `POST statuses/filter` will be helpfull for you here